# Homework 1: Intro to MLOps

## By: Lorenzo Cesconetto

Check out all the questions [here](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2025/01-intro/homework.md).


In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
import numpy as np

## Question 1: Downloading the data

We'll use [the same NYC taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?


In [2]:
df_jan = pd.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
)

In [3]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
# There are 19 columns
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
# Another way to see there are 19 columns in the dataset
print(f"Total number of columns: {len(df_jan.columns)}")

Total number of columns: 19


## Question 2: Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?


In [6]:
# Compute duration
df_jan["duration"] = df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]
df_jan["duration_in_minutes"] = df_jan["duration"].apply(lambda x: x.total_seconds() / 60)

In [7]:
# Standard Deviation is 42.59
print(
    f"Standard deviation of duration in minutes: {df_jan["duration_in_minutes"].std():.2f} minutes"
)

Standard deviation of duration in minutes: 42.59 minutes


In [8]:
# An alternative to see some extra statistics. Standard Deviation is 42.59
# Notice there are negative durations!
df_jan["duration_in_minutes"].describe(percentiles=[0.01, 0.02, 0.25, 0.5, 0.75, 0.98, 0.99])

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
1%       7.833333e-01
2%       2.050000e+00
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
98%      4.873333e+01
99%      5.725000e+01
max      1.002918e+04
Name: duration_in_minutes, dtype: float64

## Question 3: Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?


In [9]:
# Fraction left after dropping outliers is 98.1%
initial_rows = len(df_jan)
cleaned_rows = sum((1 <= df_jan["duration_in_minutes"]) & (df_jan["duration_in_minutes"] <= 60))
print(f"Fraction of non outlier trips: {cleaned_rows / initial_rows:.1%}")

Fraction of non outlier trips: 98.1%


In [10]:
# Filter dataset
df_jan = df_jan[(1 <= df_jan["duration_in_minutes"]) & (df_jan["duration_in_minutes"] <= 60)]

## Question 4: One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?


In [11]:
categorical = ["PULocationID", "DOLocationID"]

In [12]:
df_jan[categorical].dtypes

PULocationID    int64
DOLocationID    int64
dtype: object

In [13]:
df_jan[categorical] = df_jan[categorical].astype(str)

In [14]:
df_jan[categorical].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [15]:
df_jan[categorical].iloc[:10].to_dict(orient="records")

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'}]

In [16]:
train_dicts = df_jan[categorical].to_dict(orient="records")
train_dicts[0]

{'PULocationID': '161', 'DOLocationID': '141'}

In [17]:
dv = DictVectorizer(sparse=True)  # Setting to False eats up too much RAM

In [18]:
X_train = dv.fit_transform(train_dicts)

In [19]:
print(f"Dimensionality of this matrix (number of columns): {len(dv.feature_names_)}")

Dimensionality of this matrix (number of columns): 515


# Question 5: Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?


In [20]:
target = "duration_in_minutes"
y_train = df_jan[target].values

In [21]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [22]:
y_pred = lr.predict(X_train)

In [23]:
print(
    f"Root Mean Squared Error (RMSE): {root_mean_squared_error(y_true=y_train, y_pred=y_pred):.2f}"
)

Root Mean Squared Error (RMSE): 7.65


# Question 6: Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?


In [24]:
df_feb = pd.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"
)

In [25]:
# Compute duration
df_feb["duration"] = df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"]
df_feb["duration_in_minutes"] = df_feb["duration"].apply(lambda x: x.total_seconds() / 60)
# Filter dataset
df_feb = df_feb[(1 <= df_feb["duration_in_minutes"]) & (df_feb["duration_in_minutes"] <= 60)]

In [26]:
# Notice we need to convert to INT otherwise OHE won't work
df_feb[categorical]

,PULocationID,DOLocationID
0,142.0,163.0
3,132.0,26.0
4,161.0,145.0
5,148.0,236.0
6,137.0,244.0
...,...,...
2913950,249.0,140.0
2913951,186.0,79.0
2913952,158.0,143.0
2913953,79.0,162.0


In [27]:
# Converting to INT to remove the ".0"
df_feb[categorical] = df_feb[categorical].astype(int)
df_feb[categorical]

,PULocationID,DOLocationID
0,142,163
3,132,26
4,161,145
5,148,236
6,137,244
...,...,...
2913950,249,140
2913951,186,79
2913952,158,143
2913953,79,162


In [28]:
# Now we can convert to object
df_feb[categorical] = df_feb[categorical].astype(str)

In [29]:
val_dicts = df_feb[categorical].to_dict(orient="records")
val_dicts[:3]

[{'PULocationID': '142', 'DOLocationID': '163'},
 {'PULocationID': '132', 'DOLocationID': '26'},
 {'PULocationID': '161', 'DOLocationID': '145'}]

In [30]:
X_val = dv.transform(val_dicts)

In [31]:
y_val = df_feb[target].values

In [32]:
y_val_pred = lr.predict(X_val)

In [33]:
print(
    f"Validation Root Mean Squared Error (RMSE): {root_mean_squared_error(y_true=y_val, y_pred=y_val_pred):.2f}"
)

Validation Root Mean Squared Error (RMSE): 7.81
